## Notebook used to calculate spectrum of $\Phi$ values using modified PyPhi

#### First get all the concepts for the original TPM

In [17]:
import pyphi
import numpy as np
import itertools
from pyphi import Direction
from pyphi import utils
from pyphi import cache
from pyphi import models

# Transition probability matrix. Saying where each state (little-endian binary) goes
tpm = np.array([
    [0.,0.,0.],
    [0.,0.,0.],
    [1.,0.,0.],
    [1.,0.,1.],
    [0.,1.,0.],
    [0.,1.,0.],
    [1.,1.,0.],
    [1.,1.,1.]
])

# Set up network object
network = pyphi.Network(tpm, node_labels=['R','C','P'])
print("Network = ",network.node_labels)

# Put the system into a given state
state = (0,0,0)
nodes = ['R','C','P']
subsystem = pyphi.Subsystem(network, state, nodes)


## Find all concepts for a given subsystem
all_concepts = []
mechanisms = utils.powerset(subsystem.node_indices, nonempty=True)
for mechanism in mechanisms:
#     print("MECHANISM = ",mechanism)
    mechanism_concepts = []
    
    # Find the CORE cause(s) for a given mechanism
    direction = Direction.CAUSE
    past_purviews = subsystem.potential_purviews(direction, mechanism, purviews=False)
    all_causes = [subsystem.find_mip(direction, mechanism, purview) for purview in past_purviews]

    if not past_purviews:
        core_cause = [_null_ria(direction, mechanism, ())]
    else:
        core_cause = max(all_causes)
        if core_cause.phi > 0.:
            all_core_causes = [value for index,value in enumerate(subsystem.find_mip(direction, mechanism, purview) for purview in past_purviews) if value.phi == core_cause.phi]
        else:
            all_core_causes = [core_cause]
            
#     print("Core Cause(s) for Mechanism : ",all_core_causes)
    
    # Find the CORE effect(s) for a given mechanism
    direction = Direction.EFFECT
    future_purviews = subsystem.potential_purviews(direction, mechanism, purviews=False)
    all_effects = [subsystem.find_mip(direction, mechanism, purview) for purview in future_purviews]

    if not future_purviews:
        core_effect = [_null_ria(direction, mechanism, ())]
    else:
        core_effect = max(all_effects)
        if core_effect.phi > 0.:
            all_core_effects = [value for index,value in enumerate(subsystem.find_mip(direction, mechanism, purview) for purview in past_purviews) if value.phi == core_effect.phi]
        else:
            all_core_effects = [core_effect]
            
#     print("Core Effects(s) for Mechanism : ",all_core_effects)

    ## Now we need to build concepts for all possible core cause and core effect combinations
    for each_cause in all_core_causes:
        MIC = models.MaximallyIrreducibleCause(each_cause)
        for each_effect in all_core_effects:
            MIE = models.MaximallyIrreducibleEffect(each_effect)
            
            concept = models.Concept(mechanism = mechanism, cause = MIC, effect = MIE, subsystem = subsystem)
            mechanism_concepts.append(concept)
            
    all_concepts.append(mechanism_concepts)
    
    
print(all_concepts)
## Create all CES as the cartesian product of the concepts for each mechanism
all_CES = []
for element in itertools.product(*all_concepts):
    all_CES.append(models.CauseEffectStructure(concepts=element))
    
for CES in all_CES:
    print("HERE")
    print(CES)


Network =  NodeLabels(('R', 'C', 'P'))
[[━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
           Concept: Mechanism = [R], φ = 1/4            
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
            MIC                         MIE             
┌──────────────────────────┐┌──────────────────────────┐
│  φ = 1/2                 ││  φ = 1/4                 │
│  Purview = [C]           ││  Purview = [P]           │
│  MIP:                    ││  MIP:                    │
│     ∅     R              ││     ∅     R              │
│    ─── ✕ ───             ││    ─── ✕ ───             │
│     C     ∅              ││     P     ∅              │
│  Repertoire:             ││  Repertoire:             │
│    ┌─────────────┐       ││    ┌─────────────┐       │
│    │ S    Pr(S)  │       ││    │ S    Pr(S)  │       │
│    │ ╴╴╴╴╴╴╴╴╴╴╴ │       ││    │ ╴╴╴╴╴╴╴╴╴╴╴ │       │
│    │ 0    1      │       ││    │ 0    1      │       │
│    │ 1    0      │       ││    │ 1    0      

#### Now cut the original TPM and find the new set of concepts. Get the Phi value and repeat the cut

In [27]:
from pyphi import compute
from pyphi import partition
from pyphi import models

subsystem = pyphi.Subsystem(network, state, nodes,cut=None)

unpartitioned_ces = compute.subsystem.ces(subsystem)

sia_bipartitions = compute.subsystem.sia_bipartitions(subsystem.cut_indices, subsystem.cut_node_labels)


for cut in sia_bipartitions[0:1]:
    print("Evaluating Cut ",cut)
    new_subsystem = subsystem.apply_cut(cut)
    
    ## Find all concepts for a given subsystem
    all_concepts = []
    mechanisms = utils.powerset(new_subsystem.node_indices, nonempty=True)
    for mechanism in mechanisms:
#         print("MECHANISM = ",mechanism)
        mechanism_concepts = []

        # Find the CORE cause(s) for a given mechanism
        direction = Direction.CAUSE
        past_purviews = new_subsystem.potential_purviews(direction, mechanism, purviews=False)
        all_causes = [new_subsystem.find_mip(direction, mechanism, purview) for purview in past_purviews]

        if not past_purviews:
            core_cause = [_null_ria(direction, mechanism, ())]
        else:
            core_cause = max(all_causes)
            if core_cause.phi > 0.:
                all_core_causes = [value for index,value in enumerate(new_subsystem.find_mip(direction, mechanism, purview) for purview in past_purviews) if value.phi == core_cause.phi]
            else:
                all_core_causes = [core_cause]

#         print("Core Cause(s) for Mechanism : ",all_core_causes)

        # Find the CORE effect(s) for a given mechanism
        direction = Direction.EFFECT
        future_purviews = new_subsystem.potential_purviews(direction, mechanism, purviews=False)
        all_effects = [new_subsystem.find_mip(direction, mechanism, purview) for purview in future_purviews]

        if not future_purviews:
            core_effect = [_null_ria(direction, mechanism, ())]
        else:
            core_effect = max(all_effects)
            if core_effect.phi > 0.:
                all_core_effects = [value for index,value in enumerate(new_subsystem.find_mip(direction, mechanism, purview) for purview in past_purviews) if value.phi == core_effect.phi]
            else:
                all_core_effects = [core_effect]

#         print("Core Effects(s) for Mechanism : ",all_core_effects)

        ## Now we need to build concepts for all possible core cause and core effect combinations
        for each_cause in all_core_causes:
            MIC = models.MaximallyIrreducibleCause(each_cause)
            for each_effect in all_core_effects:
                MIE = models.MaximallyIrreducibleEffect(each_effect)

                concept = models.Concept(mechanism = mechanism, cause = MIC, effect = MIE, subsystem = new_subsystem)
                mechanism_concepts.append(concept)

        all_concepts.append(mechanism_concepts)
    
    
#     print(all_concepts)
    new_CES = models.CauseEffectStructure(concepts=all_concepts[0])
    print(new_CES)
    
    for CES in all_CES:
        fo
        Phi=compute.distance.ces_distance(CES,new_CES)
        print(r'$Phi$ = ',Phi)
    
#     ## Create all CES as the cartesian product of the concepts for each mechanism
#     all_CES = []
#     for element in itertools.product(*all_concepts):
#         all_CES.append(models.CauseEffectStructure(concepts=element))

#     for CES in all_CES:
#         print("HERE")
#         print(CES)

Evaluating Cut  Cut [R] ━━/ /━━➤ [C, P]
════════════════════════════════════════════════════════════
             Cause-effect structure (1 concept)             
════════════════════════════════════════════════════════════
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              Concept: Mechanism = [R], φ = 0               
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              MIC                         MIE               
  ┌──────────────────────────┐┌──────────────────────────┐  
  │  φ = 1/2                 ││  φ = 0                   │  
  │  Purview = [C]           ││  Purview = [R]           │  
  │  MIP:                    ││  MIP:                    │  
  │     ∅     R              ││     ∅     R              │  
  │    ─── ✕ ───             ││    ─── ✕ ───             │  
  │     C     ∅              ││     R     ∅              │  
  │  Repertoire:             ││  Repertoire:             │  
  │    ┌─────────────┐       ││    ┌─────────